### Manipulation of PeaskDB de novo-assisted database search results of Trocas 7 (high water, April 2019) incubation samples LC-MS/MS data using python.

Starting with:

    PeaksDB search results (.csv) of PTM-optimized database searches
    These were all searched with 15 ppm precursor tolerance and 0.5 ppm fragement ion tolerance
    Exported at <1.0% FDR
    
Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the TROCAS # (e.g., 101) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

In [1]:
cd /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/402_TROCAS7_Fusion_Apr2021_PEAKS_131/

/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/402_TROCAS7_Fusion_Apr2021_PEAKS_131


In [2]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [3]:
ls 

 402_TROCAS7_Fusion_Apr2021_PEAKS_131_peptide.csv
 402_TROCAS7_Fusion_Apr2021_PEAKS_131_protein.csv
 402_TROCAS7_Fusion_Apr2021_PEAKS_131_protein.fasta
'DB search psm.csv'
'de novo only peptides.csv'
 peptides.pep.xml
 protein-peptides.csv


In [4]:
# read the CSV into a dataframe using the pandas read_csv function
peaksdbdup402 = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/402_TROCAS7_Fusion_Apr2021_PEAKS_131/402_TROCAS7_Fusion_Apr2021_PEAKS_131_peptide.csv")

# remove redundant rows
peaksdb402 = pd.DataFrame.drop_duplicates(peaksdbdup402)

print(peaksdb402.columns)

columns = ['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area', 'Fraction', 'Scan', 'Source_File',
       '#Spec', '#Spec', 'Accession', 'PTM',
       'AScore']

peaksdb402.columns = columns

#remmove # spec and accession columns because they mess parsing up

del peaksdb402['#Spec']
del peaksdb402['Accession']
del peaksdb402['PTM']
del peaksdb402['AScore']

mean_length = peaksdb402['Length'].mean()
print(mean_length)

print("# redundant peaksdb peptides in combined dataframe", len(peaksdbdup402))
print("# nonredundant peaksdb peptides in combined dataframe", len(peaksdb402))

#look at the dataframe
peaksdb402.head()

Index(['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area Trocas7-402-Bay', 'Fraction', 'Scan', 'Source File', '#Spec',
       '#Spec Trocas7-402-Bay', 'Accession', 'PTM', 'AScore'],
      dtype='object')
12.150943396226415
# redundant peaksdb peptides in combined dataframe 53
# nonredundant peaksdb peptides in combined dataframe 53


,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,Source_File
0,SC(+57.02)AAAGTEC(+57.02)LISGWGNTK,78.58,1881.8350,18,0.2,941.9250,92.13,749000.0,5,14685,20210406_Trocas7_654_Bay402_DDA_120min_1.raw
1,LGEHNIDVLEGNEQFINAAK,67.83,2210.0967,20,0.2,737.7063,94.82,858000.0,5,15147,20210406_Trocas7_654_Bay402_DDA_120min_1.raw
2,SSGSSYPSLLQC(+57.02)LK,52.61,1525.7446,14,0.5,763.8799,100.41,159000.0,5,16097,20210406_Trocas7_654_Bay402_DDA_120min_1.raw
3,APVLSDSSC(+57.02)K,41.57,1062.5016,10,-0.4,532.2579,36.05,29300.0,5,5771,20210406_Trocas7_654_Bay402_DDA_120min_1.raw
4,LSSPATLN(+.98)SR,41.21,1045.5404,10,0.3,523.7776,50.78,495000.0,5,8259,20210406_Trocas7_654_Bay402_DDA_120min_1.raw


In [6]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaksdb402['A'] = peaksdb402['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaksdb402['C'] = peaksdb402['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaksdb402['D'] = peaksdb402['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaksdb402['E'] = peaksdb402['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaksdb402['F'] = peaksdb402['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaksdb402['G'] = peaksdb402['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaksdb402['H'] = peaksdb402['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaksdb402 output, there will be no isoleucines (they're lumped in with leucines)
peaksdb402['I'] = peaksdb402['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaksdb402['K'] = peaksdb402['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaksdb402['L'] = peaksdb402['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaksdb402['M'] = peaksdb402['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaksdb402['N'] = peaksdb402['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaksdb402['P'] = peaksdb402['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaksdb402['Q'] = peaksdb402['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaksdb402['R'] = peaksdb402['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaksdb402['S'] = peaksdb402['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaksdb402['T'] = peaksdb402['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaksdb402['V'] = peaksdb402['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaksdb402['W'] = peaksdb402['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaksdb402['Y'] = peaksdb402['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaksdb402['c-carb'] = peaksdb402['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaksdb402['m-oxid'] = peaksdb402['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaksdb402['n-deam'] = peaksdb402['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a lamba function to enumerate the # of deamidated Q's in each peptide
peaksdb402['q-deam'] = peaksdb402['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# create a column with 'stripped' peptide sequences using strip
peaksdb402['stripped_peptide'] = peaksdb402['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaksdb402['stripped_length'] = peaksdb402['stripped_peptide'].apply(len)

peaksdb402['NAAF_num.'] = peaksdb402['Area'] / peaksdb402['stripped_length']

# total the number of modifications in sequence
peaksdb402['ptm-total'] = peaksdb402['c-carb'] + peaksdb402['m-oxid'] + peaksdb402['n-deam'] + peaksdb402['q-deam']

# turn all isoleucines into leucines
# this helps later in comparing Unipept peptides to PeaksDB and Comet ones
peaksdb402['stripped_IL']= peaksdb402['stripped_peptide'].str.replace('I','L')

# write modified dataframe to new txt file
peaksdb402.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/402_BY_T24_GD_PDB/402A_BY_T24_GD_PDB.csv")

# check out the results
peaksdb402.head()

,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,...,Y,c-carb,m-oxid,n-deam,q-deam,stripped_peptide,stripped_length,NAAF_num.,ptm-total,stripped_IL
0,SC(+57.02)AAAGTEC(+57.02)LISGWGNTK,78.58,1881.8350,18,0.2,941.9250,92.13,749000.0,5,14685,...,0,2,0,0,0,SCLISGWGNTK,11,68090.909091,2,SCLLSGWGNTK
1,LGEHNIDVLEGNEQFINAAK,67.83,2210.0967,20,0.2,737.7063,94.82,858000.0,5,15147,...,0,0,0,0,0,LGEHNIDVLEGNEQFINAAK,20,42900.000000,0,LGEHNLDVLEGNEQFLNAAK
2,SSGSSYPSLLQC(+57.02)LK,52.61,1525.7446,14,0.5,763.8799,100.41,159000.0,5,16097,...,1,1,0,0,0,SSGSSYPSLLQCLK,14,11357.142857,1,SSGSSYPSLLQCLK
3,APVLSDSSC(+57.02)K,41.57,1062.5016,10,-0.4,532.2579,36.05,29300.0,5,5771,...,0,1,0,0,0,APVLSDSSCK,10,2930.000000,1,APVLSDSSCK
4,LSSPATLN(+.98)SR,41.21,1045.5404,10,0.3,523.7776,50.78,495000.0,5,8259,...,0,0,0,1,0,LSSPATLNSR,10,49500.000000,1,LSSPATLNSR


In [7]:
# keep only stripped peptide I/L and NAAF
db_402 = peaksdb402[['stripped_IL', 'Area', 'NAAF_num.']]

db_402.set_index('stripped_IL')

# write modified dataframe to new txt file
db_402.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/402A_BY_T24_GD_PDB_ILnaafs.csv")

db_402.head()

,stripped_IL,Area,NAAF_num.
0,SCLLSGWGNTK,749000.0,68090.909091
1,LGEHNLDVLEGNEQFLNAAK,858000.0,42900.000000
2,SSGSSYPSLLQCLK,159000.0,11357.142857
3,APVLSDSSCK,29300.0,2930.000000
4,LSSPATLNSR,495000.0,49500.000000


### Exporting txt files of stripped peptides at confidence cutoffs:

In [8]:
# keep only peptide column peptides <1% FDR (this is what we exported)
pep402moddup = peaksdb402[["Peptide"]]

# keep only the stripped peptide column <1% FDR
# this is what we'll use for UniPept input, etc
pep402dup = peaksdb402[["stripped_peptide"]]

# deduplicate both of these lists
pep402mod = pep402moddup.drop_duplicates()
pep402 = pep402dup.drop_duplicates()

# print out the #s of modified and stripped peptides, deduplicated and not

print("Total modified PeaksDB peptides in 402:", len(pep402moddup))
print("Deduplicated modified PeaksDB peptides in 402:", len(pep402mod))
print("Total stripped PeaksDB peptides in 402:", len(pep402dup))
print("Deduplicated stripped PeaksDB peptides in 402:", len(pep402))

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing
pep402.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/402_BY_T24_GD_PDB/402A_BY_T24_GD_PDB_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/402_BY_T24_GD_PDB/402A_BY_T24_GD_PDB_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/402_BY_T24_GD_PDB/402A_BY_T24_GD_PDB_stripped_peptides.fas

# look at the stripped peptides
pep402.head()

Total modified PeaksDB peptides in 402: 53
Deduplicated modified PeaksDB peptides in 402: 53
Total stripped PeaksDB peptides in 402: 53
Deduplicated stripped PeaksDB peptides in 402: 50


,stripped_peptide
0,SCLISGWGNTK
1,LGEHNIDVLEGNEQFINAAK
2,SSGSSYPSLLQCLK
3,APVLSDSSCK
4,LSSPATLNSR


## NAAF correction and exporting files with AA and PTM totals:

In [9]:
# made a new dataframe that contains the suMN of certain columns in the modified
# peptide dataframe above 

index = ['sample total']

data = {'A': peaksdb402['A'].sum(),
        'C': peaksdb402['C'].sum(),
        'D': peaksdb402['D'].sum(),
        'E': peaksdb402['E'].sum(),
        'F': peaksdb402['F'].sum(),
        'G': peaksdb402['G'].sum(),
        'H': peaksdb402['H'].sum(),
        'I': peaksdb402['I'].sum(),
        'K': peaksdb402['K'].sum(),
        'L': peaksdb402['L'].sum(),
        'M': peaksdb402['M'].sum(),
        'N': peaksdb402['N'].sum(),
        'P': peaksdb402['P'].sum(),
        'Q': peaksdb402['Q'].sum(),
        'R': peaksdb402['R'].sum(),
        'S': peaksdb402['S'].sum(),
        'T': peaksdb402['T'].sum(),
        'V': peaksdb402['V'].sum(),
        'W': peaksdb402['W'].sum(),
        'Y': peaksdb402['Y'].sum(),
        'c-carb': peaksdb402['c-carb'].sum(),
        'm-oxid': peaksdb402['m-oxid'].sum(),
        'n-deam': peaksdb402['n-deam'].sum(),
        'q-deam': peaksdb402['q-deam'].sum(),
        'Total area': peaksdb402['Area'].sum(),
        'Total length': peaksdb402['stripped_length'].sum()
       }

totalpeaksdb402 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                              'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                              'c-carb', 'm-oxid', 
                                              'n-deam', 'q-deam', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaksdb402['% C w/ carb'] = totalpeaksdb402['c-carb'] / totalpeaksdb402['C'] 

# calculate percentage of M's that are oxidized
totalpeaksdb402['% M w/ oxid'] = totalpeaksdb402['m-oxid'] / totalpeaksdb402['M'] 

# calculate percentage of N's that are deamidated
totalpeaksdb402['% N w/ deam'] = totalpeaksdb402['n-deam'] / totalpeaksdb402['N'] 

# calculate percentage of N's that are deamidated
totalpeaksdb402['% Q w/ deam'] = totalpeaksdb402['q-deam'] / totalpeaksdb402['Q'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaksdb402['NAAF denom.'] = totalpeaksdb402['Total area'] / totalpeaksdb402['Total length']

# write modified dataframe to new txt file
totalpeaksdb402.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/402_BY_T24_GD_PDB/402A_BY_T24_GD_PDB_totals.csv")

totalpeaksdb402.head()

,A,C,D,E,F,G,H,I,K,L,...,m-oxid,n-deam,q-deam,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF denom.
sample total,52,11,23,39,19,57,15,48,32,63,...,2,8,5,109357590.0,609,1.0,0.4,0.121212,0.227273,179569.1133


In [10]:
# use the calculated NAAF factor (in totalpeaksdb402 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

NAAF20 = 179569.1133

# we'll use -10lgP > 20 as an approximate cutoff for the <1.0% FDR which is what we actually have
peaksdb402['NAAF factor'] = (peaksdb402['NAAF_num.'])/NAAF20

# make a dataframe that contains only what we need: sequences, AAs, PTMN
peaksdb402_NAAF = peaksdb402[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', \
                              'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                              'c-carb', 'm-oxid', 'n-deam', 'q-deam']].copy()

# multiply the NAAF20 factor by the AAs to normalize its abundance by peak area and peptide length

peaksdb402_NAAF['A-NAAF20'] = peaksdb402_NAAF['A'] * peaksdb402['NAAF factor']
peaksdb402_NAAF['C-NAAF20'] = peaksdb402_NAAF['C'] * peaksdb402['NAAF factor']
peaksdb402_NAAF['D-NAAF20'] = peaksdb402_NAAF['D'] * peaksdb402['NAAF factor']
peaksdb402_NAAF['E-NAAF20'] = peaksdb402_NAAF['E'] * peaksdb402['NAAF factor']
peaksdb402_NAAF['F-NAAF20'] = peaksdb402_NAAF['F'] * peaksdb402['NAAF factor']
peaksdb402_NAAF['G-NAAF20'] = peaksdb402_NAAF['G'] * peaksdb402['NAAF factor']
peaksdb402_NAAF['H-NAAF20'] = peaksdb402_NAAF['H'] * peaksdb402['NAAF factor']
peaksdb402_NAAF['I-NAAF20'] = peaksdb402_NAAF['I'] * peaksdb402['NAAF factor']
peaksdb402_NAAF['K-NAAF20'] = peaksdb402_NAAF['K'] * peaksdb402['NAAF factor']
peaksdb402_NAAF['L-NAAF20'] = peaksdb402_NAAF['L'] * peaksdb402['NAAF factor']
peaksdb402_NAAF['M-NAAF20'] = peaksdb402_NAAF['M'] * peaksdb402['NAAF factor']
peaksdb402_NAAF['N-NAAF20'] = peaksdb402_NAAF['N'] * peaksdb402['NAAF factor']
peaksdb402_NAAF['P-NAAF20'] = peaksdb402_NAAF['P'] * peaksdb402['NAAF factor']
peaksdb402_NAAF['Q-NAAF20'] = peaksdb402_NAAF['Q'] * peaksdb402['NAAF factor']
peaksdb402_NAAF['R-NAAF20'] = peaksdb402_NAAF['R'] * peaksdb402['NAAF factor']
peaksdb402_NAAF['S-NAAF20'] = peaksdb402_NAAF['S'] * peaksdb402['NAAF factor']
peaksdb402_NAAF['T-NAAF20'] = peaksdb402_NAAF['T'] * peaksdb402['NAAF factor']
peaksdb402_NAAF['V-NAAF20'] = peaksdb402_NAAF['V'] * peaksdb402['NAAF factor']
peaksdb402_NAAF['W-NAAF20'] = peaksdb402_NAAF['W'] * peaksdb402['NAAF factor']
peaksdb402_NAAF['Y-NAAF20'] = peaksdb402_NAAF['Y'] * peaksdb402['NAAF factor']

# multiply the NAAF20 factor by the PTMN normalize its abundance by peak area and peptide length

peaksdb402_NAAF['ccarb-NAAF20'] = peaksdb402_NAAF['c-carb'] * peaksdb402_NAAF['NAAF factor']
peaksdb402_NAAF['moxid-NAAF20'] = peaksdb402_NAAF['m-oxid'] * peaksdb402_NAAF['NAAF factor']
peaksdb402_NAAF['ndeam-NAAF20'] = peaksdb402_NAAF['n-deam'] * peaksdb402_NAAF['NAAF factor']
peaksdb402_NAAF['qdeam-NAAF20'] = peaksdb402_NAAF['q-deam'] * peaksdb402_NAAF['NAAF factor']


# write the dataframe to a new csv
peaksdb402_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/402_BY_T24_GD_PDB/402A_BY_T24_GD_PDB_naaf.csv")

peaksdb402_NAAF.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,I,...,R-NAAF20,S-NAAF20,T-NAAF20,V-NAAF20,W-NAAF20,Y-NAAF20,ccarb-NAAF20,moxid-NAAF20,ndeam-NAAF20,qdeam-NAAF20
0,SCLISGWGNTK,0.379191,3,2,0,1,0,3,0,1,...,0.00000,0.758381,0.758381,0.000000,0.379191,0.000000,0.758381,0.0,0.00000,0.0
1,LGEHNIDVLEGNEQFINAAK,0.238905,2,0,1,3,1,2,1,2,...,0.00000,0.000000,0.000000,0.238905,0.000000,0.000000,0.000000,0.0,0.00000,0.0
2,SSGSSYPSLLQCLK,0.063247,0,1,0,0,0,1,0,0,...,0.00000,0.316233,0.000000,0.000000,0.000000,0.063247,0.063247,0.0,0.00000,0.0
3,APVLSDSSCK,0.016317,1,1,1,0,0,0,0,0,...,0.00000,0.048951,0.000000,0.016317,0.000000,0.000000,0.016317,0.0,0.00000,0.0
4,LSSPATLNSR,0.275660,1,0,0,0,0,0,0,0,...,0.27566,0.826980,0.275660,0.000000,0.000000,0.000000,0.000000,0.0,0.27566,0.0


In [11]:
# made a new dataframe that contains the suMN of NAAF normalized AAs for peaksdb402 peaksdb results
# also contains the suMN of the NAAF-corrected PTMN occurances for each affected residue

index = ['sample total']

data = {'NAAF': peaksdb402_NAAF['NAAF factor'].sum(),
        'A': peaksdb402_NAAF['A-NAAF20'].sum(),
        'C': peaksdb402_NAAF['C-NAAF20'].sum(),
        'D': peaksdb402_NAAF['D-NAAF20'].sum(),
        'E': peaksdb402_NAAF['E-NAAF20'].sum(),
        'F': peaksdb402_NAAF['F-NAAF20'].sum(),
        'G': peaksdb402_NAAF['G-NAAF20'].sum(),
        'H': peaksdb402_NAAF['H-NAAF20'].sum(),
        'I': peaksdb402_NAAF['I-NAAF20'].sum(),
        'K': peaksdb402_NAAF['K-NAAF20'].sum(),
        'L': peaksdb402_NAAF['L-NAAF20'].sum(),
        'M': peaksdb402_NAAF['M-NAAF20'].sum(),
        'N': peaksdb402_NAAF['N-NAAF20'].sum(),
        'P': peaksdb402_NAAF['P-NAAF20'].sum(),
        'Q': peaksdb402_NAAF['Q-NAAF20'].sum(),
        'R': peaksdb402_NAAF['R-NAAF20'].sum(),
        'S': peaksdb402_NAAF['S-NAAF20'].sum(),
        'T': peaksdb402_NAAF['T-NAAF20'].sum(),
        'V': peaksdb402_NAAF['V-NAAF20'].sum(),
        'W': peaksdb402_NAAF['W-NAAF20'].sum(),
        'Y': peaksdb402_NAAF['Y-NAAF20'].sum(),
        'c-carb': peaksdb402_NAAF['ccarb-NAAF20'].sum(),
        'm-oxid': peaksdb402_NAAF['moxid-NAAF20'].sum(),
        'n-deam': peaksdb402_NAAF['ndeam-NAAF20'].sum(),
        'q-deam': peaksdb402_NAAF['qdeam-NAAF20'].sum()
       }

totalpeaksdb402_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', \
                                                   'G', 'H', 'I','K', 'L', 'M', \
                                                   'N', 'P', 'Q', 'R', 'S', \
                                                   'T', 'V', 'W', 'Y', 'c-carb', \
                                                   'm-oxid', 'n-deam', 'q-deam'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, and Rs


# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaksdb402_NAAF['% C w/ carb.'] = totalpeaksdb402_NAAF['c-carb'] / totalpeaksdb402_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaksdb402_NAAF['% M w/ oxid'] = totalpeaksdb402_NAAF['m-oxid'] / totalpeaksdb402_NAAF['M'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb402_NAAF['% N w/ deam'] = totalpeaksdb402_NAAF['n-deam'] / totalpeaksdb402_NAAF['N'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb402_NAAF['% Q w/ deam'] = totalpeaksdb402_NAAF['q-deam'] / totalpeaksdb402_NAAF['Q'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaksdb402_NAAF['NAAF check'] = totalpeaksdb402_NAAF['NAAF'] / 179569.1133

# write modified dataframe to new txt file, same name + totals
totalpeaksdb402_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/402_BY_T24_GD_PDB/402A_BY_T24_GD_PDB_naaf_totals.csv")

totalpeaksdb402_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,Y,c-carb,m-oxid,n-deam,q-deam,% C w/ carb.,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF check
sample total,70.51187,56.702174,1.53197,1.206571,2.832897,0.632649,4.988308,0.995169,3.78019,1.360503,...,1.373147,1.53197,0.053541,0.360018,0.034798,1.0,0.158633,0.019098,0.024806,0.000393
